### Stock Buy/Sell Prediction
In this mini-project - You will be working with real stock data pulled through the alpaca.markets api. We will provide you starter code below - So you can get started with using the API. You have about 2 weeks to work on this mini-project. Please get started early as identifying good strategies can take time. Good luck and have fun in the process!

### Logistics
Please work in groups of 2 - This is a requirement for this project. Your submission will include one submission of all the training code, back testing code, plots, requirements as outlined below + breakdown of contributions by each member of the project. Please make sure each member in your team tries out new trading strategies/algorithms and does coding as a way of contributing to the project. Also working in teams is a great way to brainstorm, collaborate and improve on your winning strategies.

### Question 1
1. Pick any 5 tech stocks that have had a good performance year-over-year in the past year. You can look up NASDAQ tech stocks list or pick your favorites.
1. Download the "closing price" and "opening price" data for each day of the 5 tech stocks over the past year using the ALPACA api (see below for starter code).
1. Train at least 2 baselines (from the lecture - e.g. SMA, EMA) AND one or more ML based models (e.g. STL or supervised learning) on first 7 months of the data set
1. Back-test your buy-sell strategy on the recent past 5 months for each of the 5 stocks you picked. Assume you are not doing day-trading - So you can't sell the stock the same day you bought them. Also assume that you are buying at closing time but can sell at opening or closing time. 
1. Implement your back-testing as an automatic trading mechanism (Algorithmic Trading) - I.e. buy when you have cash available if your algorithm says buy. And sell a bought stock if your algorithm says sell. Make sure, you have a profit margin of at least 0.5%
1. Also implement your back-testing across the 5 stocks - I.e. buy/sell which ever stock is more appealing to your algorithm/trading strategy.
1. Once you initiate the back-test - Your algorithm should be able to automatically buy and sell multiple times across the back-testing period (5 months). This is an example of Algorithmic trading - Where you don't have to initiate buy/sell order but the algorithm would do it for you. Avoid a buy-sell period that contains an Earnings day in it. E.g. GOOG has earnings day on Feb 1 - Avoid a buy before Feb 1 that gets sold after Feb 1. This is to not have your backtest be skewed by wind-falls or heavy losses at earnings.
1. Use classes and modules to organize your training, back-testing, paper-trading and plotting code - See starter code for an example of methods to do paper-trading. 
1. Plot the time-series of the stocks over past 5 months and indicate using markers where your algorithm buys and sells
1. What's the total profit of your strategies. Create a table with strategy and total profit and also percentage profit as your columns. Which is your best strategy ? 
1. What other evaluation metrics make sense for backtest?
1. Also mention the insights/take-aways you gaind by working through this mini-project.

### Question 2
This one is on live paper trading. Take one of your best strategies from Question 1 and run it for 7-10 days (run it live) on the 5 stocks you picked. You can use the buy/sell order methods mentioned in starter code as an example to place orders for live paper trading. So your algorithm is automatically trading live but only on paper. Make sure, the settings for your algorithm enable you to buy and sell within the week (i.e. set your profit margin lower for sell). What's the profit you made?

### Question 3
We will also likely have a Kaggle contest a week into this mini-project where you can test your best back-trading strategies on new stocks - More on this next week!

### Starting points
1. Checkout alpaca.markets and create an account for yourself
1. Generate a key and pass for you to query alpaca.markets for paper trading
1. Checkout the colab interface that Alpaca has for you and what functionalities it has to offer. See example: https://alpaca.markets/learn/google-colab-instant-development-environments/
1. Checkout the docs for Alpaca api
1. Starter code below to get you started 

### Starter Code

In [3]:
import time 
import alpaca_trade_api as alpaca 
ENDPOINT="https://paper-api.alpaca.markets"
API_KEY_ID="PKGO8R4XDI6CB1PJKMEB" # Put in yours here - Needed for paper trading
SECRET_KEY="o0rcGgHFipTYtAAaMvwoycaqgEN6sMVPX5WpfwM7" # Put in yours here - Needed for paper trading

class TradingStrategy:
    def __init__(self,STOCK):
        self.api = alpaca.REST(API_KEY_ID, SECRET_KEY, ENDPOINT)
        self.STOCK = STOCK
        self.SELL_LIMIT_FACTOR = 1.01 # 1 percent margin
        
        # Anything else you want to initialize or a method you want to
        # call during initialization of class - Feel free to add
        
        # Get past 90 days closing prices
        self.get_past90_closing_prices()
        
    
    def get_past90_closing_prices(self):  
        barset = self.api.get_barset(self.STOCK, 'day', limit=90)
        bars = barset[self.STOCK]
        self.past90_closing_prices = [bars[index].c for index in range(len(bars))]
    
    def get_current_price(self):
        return float(self.api.get_last_trade(self.STOCK).price)
    
    def get_quantity_buy(self):
        if int(float(self.api.get_account().cash)) > 0:
            return int((float(self.api.get_account().cash)/2) \
                       /self.get_current_price())
        else:
            return 0
        
    def exists_buy_order(self):
        # Identifies if a buy order exists for a stock
        orders = self.api.list_orders()
        for order in orders:
            if order.side=="buy" and order.symbol==self.STOCK:
                return True
        
        return False
    
    def have_bought_stock(self):
        positions=self.api.list_positions()
        for position in positions:
            if position.symbol==self.STOCK and int(position.qty)==self.NEW_QUANTITY + self.EXISTING_QUANTITY:
                return True
        return False
        
        
    def get_buy_price(self):
        # Identify the buying price for a stock
        positions=self.api.list_positions()
        for position in positions:
            if position.symbol==self.STOCK:
                return float(position.cost_basis)/int(position.qty)
    
    
    def buy_market_order(self):
        # Buy the stock at market price (This is for paper-trading)
        if self.NEW_QUANTITY > 0:
            self.api.submit_order(self.STOCK, \
                        qty=self.NEW_QUANTITY,\
                        side="buy",\
                        type="market", \
                        time_in_force="day",
                        order_class=None)
        
    def buy_limit_order(self,base_price):
        pass
        
    def sell_limit_order(self):
        # (This is for paper-trading)
        pass
        # Your code if you want to sell at limit
        # Check Alpaca docs on selling at limit
        
    def identify_strategy_for_selling(self):
        # If you have multiple strategies
        # Pick between them here - Or use ML to help identify 
        # your strategy
        pass
        
        
    def market_buy_strategy(self):
        # Providing a simple trading strategy here:
        # Buy at market price if conditions are favorable for buying
        # Sell at a limit price that is determined based on buying price
        # This strategy doesn't use any ML here - You may want to use
        # appropriate libraries to train models + use the trained strategy 
        # here
        
        # Get existing quantity
        positions = self.api.list_positions()
        self.EXISTING_QUANTITY = 0
        for position in positions:
            if position.symbol == self.STOCK:
                self.EXISTING_QUANTITY += int(position.qty)
                
        # MARKET BUY order
        self.NEW_QUANTITY=self.get_quantity_buy()
        
        if self.NEW_QUANTITY == 0:
            return "ZERO EQUITY"
        
        if not self.exists_buy_order():
            self.buy_market_order()
            
        
        # BRACKET SELL order
        # Initiate sell order if stock has been bought
        # If not, wait for it to be bought
        while not self.have_bought_stock():
            #print(self.api.positions)
            #print(self.NEW_QUANTITY + self.EXISTING_QUANTITY)
            time.sleep(1)
        
        if self.have_bought_stock():
            buy_price=self.get_buy_price()
            self.SELL_LIMIT_PRICE=int(float(buy_price))*self.SELL_LIMIT_FACTOR
            
            # Initiate Sell order
            self.sell_limit_order()
      
    def your_best_strategy(self):
        # Implement here or add other methods to do the same
        pass
        
 

In [4]:
# Get an instance of your class 
strategy = TradingStrategy('AAPL')
strategy.market_buy_strategy() # Initiates a market buy and limit sell order
# This will accept a order only during market hours. However you can 
# use the market data api outside hours

/var/folders/nl/jzl2sdyd5g7gv2bk8lfg29tr0000gn/T/ipykernel_5747/1585828824.py:21: DeprecatedWarning: get_barset is deprecated as of v1.0.0. Use get_bars instead
  barset = self.api.get_barset(self.STOCK, 'day', limit=90)
/var/folders/nl/jzl2sdyd5g7gv2bk8lfg29tr0000gn/T/ipykernel_5747/1585828824.py:26: DeprecatedWarning: get_last_trade is deprecated as of v1.0.0. Use get_latest_trade instead
  return float(self.api.get_last_trade(self.STOCK).price)


KeyboardInterrupt: 

In [1]:
a = [1,2]
len(a)

2